In [ ]:
import pandas as pd

In [ ]:
# Charger les données depuis le fichier CSV
df = pd.read_csv('predictions_with_relert.csv')

In [ ]:
# Définir les groupes de fonctions selon Function (anciennement FID)
function_groups = {
    '1-5': list(range(1, 6)),
    '6-9': list(range(6, 10)),
    '10-14': list(range(10, 15)),
    '15-19': list(range(15, 20)),
    '20-24': list(range(20, 25))
}

In [ ]:
# Fonction d'assignation du groupe de fonctions
def assign_function_group(fid):
    for group_name, functions in function_groups.items():
        if fid in functions:
            return group_name
    return None

In [ ]:
# Appliquer l'assignation (utiliser 'Function' au lieu de 'FID')
df['function_group'] = df['Function'].apply(assign_function_group)

In [ ]:
# Supprimer les lignes sans groupe de fonction
df = df.dropna(subset=['function_group'])

In [ ]:
# Calculer la moyenne de RELERT par groupe (sans Algorithm)
grouped = df.groupby(['Dimension', 'function_group'])['RELERT'].mean().reset_index()

In [ ]:
# Transformer le tableau (pas de pivot car plus de colonnes à pivoter)
pivot_result = grouped.set_index(['Dimension', 'function_group']).sort_index()

In [ ]:
# Ajouter une ligne "ALL" pour chaque Dimension
all_per_dimension = (
    pivot_result
    .groupby(level=0)
    .mean()
    .assign(function_group='ALL')
    .set_index('function_group', append=True)
)

In [ ]:
# # Ajouter la ligne "ALL / ALL" globale
# all_overall = pd.DataFrame(
#     pivot_result.mean().to_dict(),
#     index=[('ALL', 'ALL')]
# )

In [ ]:
# Fusionner les résultats
final_result = pd.concat([pivot_result, all_per_dimension, all_overall])
final_result = final_result.sort_index()

In [ ]:
# Sauvegarder dans un fichier Excel
final_result.to_excel('FFNew_RELERt_summary.xlsx')
print("Tableau enregistré dans 'FFFNew_RELERt_summary.xlsx'")